In [1]:
!pip install finta
!pip install findspark

In [3]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("raw-dataset")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .getOrCreate())
sc = spark.sparkContext

22/01/24 13:17:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.options(header='True',inferSchema='True',delimiter=',').csv('/home/jovyan/work/data/dataset/raw.csv')
df = df.drop(df[0]) #drop index column (first column)

In [8]:
df = spark.read.options(header='True',inferSchema='True',delimiter=',').csv('/home/jovyan/work/data/stock_data_final')
df = df.drop(df[0]) #drop index column (first column)

In [14]:
df.groupBy("pred_3_3p").count().show()

+---------+-------+
|pred_3_3p|  count|
+---------+-------+
|     down| 167048|
|       sw|1212490|
|       up| 197765|
+---------+-------+



In [5]:
##reanme column to match Finta API require
import pyspark.sql.functions as F
columns = {"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'}

def rename_columns(df, columns):
    if isinstance(columns, dict):
        return df.select(*[F.col(col_name).alias(columns.get(col_name, col_name)) for col_name in df.columns])
    else:
        raise ValueError("'columns' should be a dict, like {'old_name_1':'new_name_1', 'old_name_2':'new_name_2'}")

df = rename_columns(df,columns)

In [6]:
from pyspark.sql.functions import to_timestamp
df = df.withColumn("Date",to_timestamp(df.Date))

In [7]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- Symbol: string (nullable = true)



In [8]:
def _exponential_smooth(key, data):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    symbol = data.Symbol.unique()[0]
    data.set_index("Date",inplace=True)
    df = data.ewm(alpha=0.65).mean()
    df.reset_index(inplace=True)
    df.loc[:,'Symbol'] = symbol
    
    return pd.DataFrame(df.values)

In [9]:
#Schema have to match with return dataframe. Check group.columns, some of indicators maybe return null 
SCHEMA_EXP= "Date string, open double, high double, low double, close double, Adj_close double, volume long, Symbol string"


df_exp = df.groupBy("Symbol").applyInPandas(_exponential_smooth,schema=SCHEMA_EXP)

In [10]:
def _get_indicator_grouped_data(key, group):
    from finta import TA
    INDICATORS = ['RSI', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV','WILLIAMS','ADX', 'TRIX']
    
    #df = pd.DataFrame()
    ind = pd.DataFrame()
    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(group)')
        if not isinstance(ind_data,pd.DataFrame):
            ind_data = ind_data.to_frame()
            group = group.merge(ind_data, left_index=True, right_index=True)

    del (group['open'])
    del (group['high'])
    del (group['low'])
    del (group['volume'])
    del (group['Adj_close'])
    
    #uncomment 2 lines below to test return column
    #print(group.columns)
    #return pd.DataFrame([key]) 
    
    return pd.DataFrame(group.values)

#SCHEMA = "key string"  test schema
        

In [11]:
#Schema have to match with return dataframe. Check group.columns, some of indicators maybe return null 
SCHEMA = "Date string, close double, symbol string, 14_period_RSI double, \
14_period_STOCH_K double, MFV double, 14_period_ATR double, MOM double, 14_period_MFI double, \
ROC double, OBV double, 20_period_CCI double, 14_period_EMV double, Williams double, 14_period_ADX double, 20_period_TRIX double"


In [12]:
df_indi = df_exp.groupBy("Symbol").applyInPandas(_get_indicator_grouped_data,schema=SCHEMA)

In [13]:
def _produce_prediction(group):
    """
    Function that produces the 'truth' values
    At a given row, it looks 'day' rows ahead to see if the price increased (up) or decreased (down).
    When the price change less than p%, it's sideways (sw).
    :param day: number of days, or rows to look ahead to see what the price did
    """
    
    
    #day = [3,5,7,10]
    #for d in day:
    #Wrong calculation when i use for loop and i have no idea
    
    p = 0.05
    pred_3 =  1 - ( group["close"] - group.shift(-3)["close"])/group["close"] 
    pred_3 = pred_3.iloc[:-3]
    pred_3 = pd.DataFrame(["down" if x < (1-p) else ("up" if x > (1+p) else "sw") for x in pred_3])
    group.loc[:,'pred_3_5p'] = pred_3
    
    pred_5 =  1 - ( group["close"] - group.shift(-5)["close"])/group["close"] 
    pred_5 = pred_5.iloc[:-5]
    pred_5 = pd.DataFrame(["down" if x < (1-p) else ("up" if x > (1+p) else "sw") for x in pred_5])
    group.loc[:,'pred_5_5p'] = pred_5
    
    pred_7 =  1 - ( group["close"] - group.shift(-7)["close"])/group["close"] 
    pred_7 = pred_7.iloc[:-7]
    pred_7 = pd.DataFrame(["down" if x < (1-p) else ("up" if x > (1+p) else "sw") for x in pred_7]) 
    group.loc[:,'pred_7_5p'] = pred_7
    
    pred_10 =  1 - ( group["close"] - group.shift(-10)["close"])/group["close"] 
    pred_10 = pred_10.iloc[:-10]
    pred_10 = pd.DataFrame(["down" if x < (1-p) else ("up" if x > (1+p) else "sw") for x in pred_10])
    group.loc[:,'pred_10_5p'] = pred_10


    group.dropna(inplace=True)
   
    return pd.DataFrame(group.values)
    #return group

#data = _produce_prediction(group)
#del (data['close'])
#data = data.dropna() # Some indicators produce NaN values for the first few rows, we just remove them here
#data.tail()

In [15]:
#Schema have to match with return dataframe. Check group.columns, some of indicators maybe return null 
SCHEMA_PREDICTION = "Date string, close double, symbol string, 14_period_RSI double, \
14_period_STOCH_K double, MFV double, 14_period_ATR double, MOM double, 14_period_MFI double, \
ROC double, OBV double, 20_period_CCI double, 14_period_EMV double, Williams double, 14_period_ADX double, 20_period_TRIX double,\
pred_3 string, pred_5 string, pred_7 string, pred_10 string"

df_prediction = df_indi.groupBy("Symbol").applyInPandas(_produce_prediction,schema=SCHEMA_PREDICTION)

In [16]:
df_prediction.show()

+----------+------------------+------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+--------+-------------------+--------------------+-------------------+------------------+-------------------+------+------+------+-------+
|      Date|             close|symbol|     14_period_RSI| 14_period_STOCH_K|               MFV|      14_period_ATR|                MOM|     14_period_MFI|                ROC|     OBV|      20_period_CCI|       14_period_EMV|           Williams|     14_period_ADX|     20_period_TRIX|pred_3|pred_5|pred_7|pred_10|
+----------+------------------+------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+--------+-------------------+--------------------+-------------------+------------------+-------------------+------+------+------+-------+
|1987-08-18|  5.94867562943439|  ABMD| 70.97639263246325| 66.

In [17]:
df_prediction.coalesce(1).write.csv ('/home/jovyan/work/data/stock_data_7p',header=True)